In [3]:
# General
import os
import warnings
import numpy as np
import pandas as pd
import re

# Data Preps
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

# Stopwords removal and stemming
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from hunspell import Hunspell

# Oversasmpling
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import ADASYN

# FastText
import fasttext
import fasttext.util

# LSTM with keras
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.models import Sequential
from keras.optimizers import Adam

# Evaluation metrics
from sklearn.metrics import confusion_matrix, classification_report

warnings.filterwarnings("ignore", category=DeprecationWarning, module="tensorflow")
warnings.filterwarnings("ignore", category=UserWarning, message="You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy.")

In [4]:
# Load data
filepath = "../dataset/final_dataset.xlsx"

df = pd.read_excel(filepath, sheet_name="10k")
X = df['tweet']
y = df['label']

# Import Pre-trained fastText model on Indonesian language

In [5]:
# Load pre-trained fasttext model on Indonesia language
filepath = "../../fasttext_pretrained_model_indonesia/cc.id.300.bin"

fasttext_model = fasttext.load_model(filepath)

# Text Preprocessing

In [6]:
def case_folding(df, column_name):
    new_df = df.copy()
    new_df[column_name] = new_df[column_name].str.lower()
    
    return new_df

In [7]:
def remove_punctuation(df, column_name):
    new_df = df.copy()
    new_df[column_name] = new_df[column_name].str.replace(r'[^\w\s]', '', regex=True)
    
    return new_df

def remove_symbols(df, column_name):
    new_df = df.copy()
    new_df[column_name] = new_df[column_name].apply(lambda text: re.sub(r'[^\x00-\x7F]+', '', text))
    
    return new_df

def remove_punctuation_and_sc(df, column_name):
    new_df = df.copy()
    new_df[column_name] = new_df[column_name].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
    
    return new_df

def remove_mentions_hashtags(df, column_name):
    new_df = df.copy()
    new_df[column_name] = new_df[column_name].apply(lambda x: re.sub(r'@\w+|\#\w+', '', x))
    
    return new_df

def remove_hyperlink(df, column_name):
    new_df = df.copy()
    new_df[column_name] = new_df[column_name].apply(lambda x: re.sub(r'http\S+', '', x))
    
    return new_df

def remove_redundant_whitespace(df, column_name):
    new_df = df.copy()
    new_df[column_name] = new_df[column_name].str.replace(r'\s+', ' ', regex=False).str.strip()
    
    return new_df

In [8]:
factory = StopWordRemoverFactory()
stopwords_sastrawi = factory.get_stop_words()

def do_stopwords_sastrawi(text):
    words = text.split()
    words_filtered = [word for word in words if not word in stopwords_sastrawi]
    return " ".join(words_filtered)

def remove_stopwords(df, column_name):
    new_df = df.copy()
    new_df[column_name] = new_df[column_name].apply(do_stopwords_sastrawi)
    
    return new_df

In [9]:
# Load the Indonesian stopwords from Hunspell
current_directory = os.getcwd()
filepath = os.path.join(current_directory, "..", "hunspell-id-main", "id_ID")

h = Hunspell(filepath,filepath)

def word_hunspell(word):
    try:
        stems = h.stem(word)
    except UnicodeEncodeError:
        stems = [word]
    
    if len(stems) == 0:
        output = word
    else:
        output = stems[0]
    return output

def stem_hunspell(text):
    hs_stem = [word_hunspell(word) for word in text.split()]
    output = ' '.join(hs_stem) 
    return output

def stemming(df, column_name):
    new_df = df.copy()
    new_df[column_name] = new_df[column_name].apply(stem_hunspell)
    
    return new_df

In [10]:
def preprocessing(df, column_name):  
    new_df = df.copy()
    new_df = remove_mentions_hashtags(new_df, column_name)
    new_df = remove_hyperlink(new_df, column_name)
    new_df = remove_punctuation_and_sc(new_df, column_name)
    new_df = case_folding(new_df, column_name)
    new_df = remove_stopwords(new_df, column_name)
    new_df = remove_redundant_whitespace(new_df, column_name)
    new_df = stemming(new_df, column_name)
    
    return new_df

In [11]:
#df_cf = case_folding(df, 'tweet')
#df_rsw = remove_stopwords(df, 'tweet')
#df_stem = stemming(df, 'tweet')
#df_rpunc = remove_punctuation(df, 'tweet')
#df_rsym = remove_symbols(df, 'tweet')
#df_rpsc = remove_punctuation_and_sc(df, 'tweet')
#df_rrw = remove_redundant_whitespace(df, 'tweet')
#df_rmh = remove_mentions_hashtags(df, 'tweet')
#df_rhl = remove_hyperlink(df, 'tweet')
#df_preprocessed = preprocessing(df, 'tweet')

# LSTM

In [12]:
# Preparing tweets and label for each respective preprocessing steps (to be compared by using evaluation metrics)

#X = df['tweet']
#y = df['label']

#X = df_cf['tweet']
#y = df_cf['label']

#X = df_rsw['tweet']
#y = df_rsw['label']

#X = df_stem['tweet']
#y = df_stem['label']

#X = df_rpunc['tweet']
#y = df_rpunc['label']

#X = df_rsym['tweet']
#y = df_rsym['label']

#X = df_rpsc['tweet']
#y = df_rpsc['label']

#X = df_rrw['tweet']
#y = df_rrw['label']

#X = df_rmh['tweet']
#y = df_rmh['label']

#X = df_rhl['tweet']
#y = df_rhl['label']

#X = df_preprocessed['tweet']
#y = df_preprocessed['label']

## No Imbalanced Class Treatment

In [13]:
# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)

In [14]:
# Tokenize the data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [15]:
# Pad the sequences
max_length = max([len(s.split()) for s in X])
X_train = pad_sequences(X_train, maxlen=max_length)
X_test = pad_sequences(X_test, maxlen=max_length)

In [16]:
# check the dimension of the embeddings
dim = fasttext_model.get_dimension()

# create the embedding matrix
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = fasttext_model.get_word_vector(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [17]:
pd.DataFrame(embedding_matrix)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.002437,-0.043058,-0.018707,0.128495,-0.018542,-0.101053,-0.034554,0.143302,-0.066489,-0.113288,...,-0.069537,-0.042619,-0.024802,-0.008546,-0.021058,-0.034511,0.009614,-0.020076,-0.057329,0.047373
2,0.006226,-0.039082,-0.188137,0.121136,-0.001375,-0.017565,0.021986,0.010326,-0.003909,-0.072086,...,0.007077,-0.125213,0.051104,-0.055601,-0.034176,-0.043827,-0.025502,-0.045835,-0.027698,0.121965
3,0.051436,-0.028671,-0.022541,0.191665,-0.029133,-0.184469,0.045916,0.121244,0.051155,-0.261428,...,0.032695,-0.007114,0.101565,0.101271,0.069635,-0.078312,0.066119,-0.240654,-0.107772,0.228045
4,-0.039191,-0.041498,-0.063466,0.100154,-0.048755,-0.259327,-0.061933,-0.004503,0.026468,-0.137056,...,-0.026037,0.006308,0.079056,-0.005207,0.022232,-0.019059,-0.042259,-0.080854,-0.021680,0.169146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23680,0.011723,0.061613,-0.031122,0.046441,0.031312,-0.056365,-0.029583,0.016447,0.030902,0.011736,...,0.008631,-0.019576,-0.059615,0.000401,-0.001603,-0.054475,0.006014,0.052755,-0.002003,0.057023
23681,-0.025238,0.014615,0.002253,0.030075,0.011921,-0.003852,-0.050046,-0.008946,-0.045779,-0.030470,...,-0.029670,0.030701,-0.035793,-0.002349,0.016751,0.002381,0.027547,-0.018048,-0.048202,-0.011137
23682,-0.011442,-0.030182,0.025152,0.030010,-0.036651,0.007234,-0.007530,-0.041400,0.014827,-0.066604,...,-0.016342,0.007197,-0.002522,-0.029678,-0.007912,-0.007289,0.048508,-0.065911,0.002160,0.013772
23683,-0.024868,0.088420,0.243243,0.129497,-0.136866,0.021089,-0.103722,0.031331,0.137365,-0.115415,...,-0.090035,0.075169,-0.143936,0.024632,-0.108176,0.055342,-0.028520,-0.065496,0.151506,0.125149


In [18]:
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, dim, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy')
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)



Epoch 1/10

102/102 [==============================] - 11s 85ms/step - loss: 0.4639 - val_loss: 0.3843
Epoch 2/10
102/102 [==============================] - 10s 94ms/step - loss: 0.3349 - val_loss: 0.3206
Epoch 3/10
102/102 [==============================] - 10s 99ms/step - loss: 0.3028 - val_loss: 0.3185
Epoch 4/10
102/102 [==============================] - 10s 99ms/step - loss: 0.2787 - val_loss: 0.3194
Epoch 5/10
102/102 [==============================] - 10s 100ms/step - loss: 0.2600 - val_loss: 0.2770
Epoch 6/10
102/102 [==============================] - 10s 100ms/step - loss: 0.2482 - val_loss: 0.2658
Epoch 7/10
102/102 [==============================] - 10s 101ms/step - loss: 0.2380 - val_loss: 0.2832
Epoch 8/10
102/102 [==============================] - 10s 101ms/step - loss: 0.2363 - val_loss: 0.3152
Epoch 9/10
102/102 [==============================] - 10s 100ms/step - loss: 0.2271 - val_loss: 0.2676
Epoch 10/10
102/102 [==============================] - 10s 101ms/step - lo

In [19]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

64/64 [==============================] - 1s 10ms/step
[[1551   83]
 [  98  294]]
              precision    recall  f1-score   support

           0       0.94      0.95      0.94      1634
           1       0.78      0.75      0.76       392

    accuracy                           0.91      2026
   macro avg       0.86      0.85      0.85      2026
weighted avg       0.91      0.91      0.91      2026



In [20]:
y_pred = model.predict(X_train)
y_pred = np.round(y_pred)
print(confusion_matrix(y_train, y_pred))
print(classification_report(y_train, y_pred))

254/254 [==============================] - 3s 10ms/step
[[6298  229]
 [ 422 1153]]
              precision    recall  f1-score   support

           0       0.94      0.96      0.95      6527
           1       0.83      0.73      0.78      1575

    accuracy                           0.92      8102
   macro avg       0.89      0.85      0.87      8102
weighted avg       0.92      0.92      0.92      8102



In [21]:
model_name = './suicide_detection_no_treatment.h5'
model.save(model_name)

c:\Users\apeir\AppData\Local\pypoetry\Cache\virtualenvs\text-suicide-ideation-detection-yF477YnD-py3.9\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Class Weighting

In [22]:
# Get the class labels and their frequencies in the training data
class_labels = np.unique(y_train)
class_freq = compute_class_weight(class_weight='balanced', classes=class_labels, y=y_train)

# Calculate the inverse frequency as the class weight
class_weight = dict(zip(class_labels, class_freq))

In [23]:
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, dim, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy')
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2, class_weight=class_weight)

Epoch 1/10
102/102 [==============================] - 14s 107ms/step - loss: 0.5535 - val_loss: 0.4558
Epoch 2/10
102/102 [==============================] - 11s 104ms/step - loss: 0.4280 - val_loss: 0.5752
Epoch 3/10
102/102 [==============================] - 11s 106ms/step - loss: 0.3855 - val_loss: 0.4056
Epoch 4/10
102/102 [==============================] - 11s 108ms/step - loss: 0.3505 - val_loss: 0.4096
Epoch 5/10
102/102 [==============================] - 11s 106ms/step - loss: 0.3477 - val_loss: 0.3977
Epoch 6/10
102/102 [==============================] - 11s 108ms/step - loss: 0.3252 - val_loss: 0.3452
Epoch 7/10
102/102 [==============================] - 11s 107ms/step - loss: 0.3157 - val_loss: 0.3741
Epoch 8/10
102/102 [==============================] - 11s 106ms/step - loss: 0.3066 - val_loss: 0.4208
Epoch 9/10
102/102 [==============================] - 11s 107ms/step - loss: 0.3032 - val_loss: 0.2972
Epoch 10/10
102/102 [==============================] - 11s 107ms/step - l

In [24]:
# evaluate the model
y_pred = model.predict(X_test)
y_pred = np.round(y_pred)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

64/64 [==============================] - 1s 10ms/step
[[1492  142]
 [  72  320]]
              precision    recall  f1-score   support

           0       0.95      0.91      0.93      1634
           1       0.69      0.82      0.75       392

    accuracy                           0.89      2026
   macro avg       0.82      0.86      0.84      2026
weighted avg       0.90      0.89      0.90      2026



In [25]:
y_pred = model.predict(X_train)
y_pred = np.round(y_pred)
print(confusion_matrix(y_train, y_pred))
print(classification_report(y_train, y_pred))

254/254 [==============================] - 3s 10ms/step
[[6095  432]
 [ 264 1311]]
              precision    recall  f1-score   support

           0       0.96      0.93      0.95      6527
           1       0.75      0.83      0.79      1575

    accuracy                           0.91      8102
   macro avg       0.86      0.88      0.87      8102
weighted avg       0.92      0.91      0.92      8102



In [26]:
model_name = './suicide_detection_cw_final.h5'
model.save(model_name)

c:\Users\apeir\AppData\Local\pypoetry\Cache\virtualenvs\text-suicide-ideation-detection-yF477YnD-py3.9\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Oversampling

### ADASYN

In [27]:
adasyn = ADASYN()
X_adasyn, y_adasyn = adasyn.fit_resample(X_train,y_train)

In [28]:
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, dim, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy')
model.fit(X_adasyn, y_adasyn, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
161/161 [==============================] - 20s 109ms/step - loss: 0.5680 - val_loss: 0.6606
Epoch 2/10
161/161 [==============================] - 18s 109ms/step - loss: 0.4573 - val_loss: 1.0240
Epoch 3/10
161/161 [==============================] - 14s 89ms/step - loss: 0.4235 - val_loss: 0.6154
Epoch 4/10
161/161 [==============================] - 14s 87ms/step - loss: 0.3930 - val_loss: 0.4192
Epoch 5/10
161/161 [==============================] - 14s 86ms/step - loss: 0.3686 - val_loss: 0.5737
Epoch 6/10
161/161 [==============================] - 14s 85ms/step - loss: 0.3539 - val_loss: 0.4893
Epoch 7/10
161/161 [==============================] - 14s 86ms/step - loss: 0.3352 - val_loss: 0.3843
Epoch 8/10
161/161 [==============================] - 14s 85ms/step - loss: 0.3217 - val_loss: 0.6602
Epoch 9/10
161/161 [==============================] - 14s 85ms/step - loss: 0.3081 - val_loss: 0.5135
Epoch 10/10
161/161 [==============================] - 14s 86ms/step - loss: 0.2

In [29]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

64/64 [==============================] - 1s 8ms/step
[[1490  144]
 [ 102  290]]
              precision    recall  f1-score   support

           0       0.94      0.91      0.92      1634
           1       0.67      0.74      0.70       392

    accuracy                           0.88      2026
   macro avg       0.80      0.83      0.81      2026
weighted avg       0.88      0.88      0.88      2026



In [30]:
y_pred = model.predict(X_train)
y_pred = np.round(y_pred)
print(confusion_matrix(y_train, y_pred))
print(classification_report(y_train, y_pred))

  1/254 [..............................] - ETA: 5s

254/254 [==============================] - 2s 8ms/step
[[6265  262]
 [ 440 1135]]
              precision    recall  f1-score   support

           0       0.93      0.96      0.95      6527
           1       0.81      0.72      0.76      1575

    accuracy                           0.91      8102
   macro avg       0.87      0.84      0.86      8102
weighted avg       0.91      0.91      0.91      8102



In [31]:
model_name = './suicide_detection_adasyn.h5'
model.save(model_name)

c:\Users\apeir\AppData\Local\pypoetry\Cache\virtualenvs\text-suicide-ideation-detection-yF477YnD-py3.9\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### SMOTEENN

In [32]:
smoteenn = SMOTEENN()
X_smoteenn, y_smoteenn = smoteenn.fit_resample(X_train,y_train)

In [33]:
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, dim, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy')
model.fit(X_smoteenn, y_smoteenn, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
117/117 [==============================] - 13s 92ms/step - loss: 0.5589 - val_loss: 0.5800
Epoch 2/10
117/117 [==============================] - 10s 89ms/step - loss: 0.4443 - val_loss: 0.3770
Epoch 3/10
117/117 [==============================] - 10s 90ms/step - loss: 0.3914 - val_loss: 0.5205
Epoch 4/10
117/117 [==============================] - 11s 90ms/step - loss: 0.3688 - val_loss: 0.5025
Epoch 5/10
117/117 [==============================] - 11s 91ms/step - loss: 0.3410 - val_loss: 0.8067
Epoch 6/10
117/117 [==============================] - 11s 91ms/step - loss: 0.3313 - val_loss: 0.4556
Epoch 7/10
117/117 [==============================] - 11s 91ms/step - loss: 0.3087 - val_loss: 0.2950
Epoch 8/10
117/117 [==============================] - 11s 91ms/step - loss: 0.2872 - val_loss: 0.1909
Epoch 9/10
117/117 [==============================] - 11s 91ms/step - loss: 0.2745 - val_loss: 0.2370
Epoch 10/10
117/117 [==============================] - 11s 92ms/step - loss: 0.259

In [34]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

64/64 [==============================] - 1s 9ms/step
[[1202  432]
 [  46  346]]
              precision    recall  f1-score   support

           0       0.96      0.74      0.83      1634
           1       0.44      0.88      0.59       392

    accuracy                           0.76      2026
   macro avg       0.70      0.81      0.71      2026
weighted avg       0.86      0.76      0.79      2026



In [35]:
y_pred = model.predict(X_train)
y_pred = np.round(y_pred)
print(confusion_matrix(y_train, y_pred))
print(classification_report(y_train, y_pred))

  1/254 [..............................] - ETA: 6s

254/254 [==============================] - 2s 9ms/step
[[5460 1067]
 [ 162 1413]]
              precision    recall  f1-score   support

           0       0.97      0.84      0.90      6527
           1       0.57      0.90      0.70      1575

    accuracy                           0.85      8102
   macro avg       0.77      0.87      0.80      8102
weighted avg       0.89      0.85      0.86      8102

